In [76]:
import numpy as np
import cv2
import pandas as pd
import os
from keras.applications import InceptionResNetV2
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Activation
from keras.layers import Input
from keras.optimizers import Adam

In [145]:
cnn = InceptionResNetV2(include_top = False, weights='imagenet', pooling='max')

In [36]:
def read_images(path, end_index = -1, start_index = 0):
    colour_images = []
    index = start_index
    for file in os.listdir(path):

        if index == end_index:
            break
        else:
            index += 1

        img = cv2.imread(path+file)
        img = cv2.resize(img, dsize=(299, 299))
        colour_images.append(img)

    colour_images = np.array(colour_images)
    return colour_images

In [163]:
def normalize_images(array):
    (length, height, width, channels) = array.shape
    array.astype(dtype=np.float32, copy=False)
    mean = array.sum(axis=0) / length
    array = np.subtract(array, mean, casting = 'same_kind')
    variance = (array ** 2).sum(axis=0) / length + 1e-27
    array = np.divide(array, variance, casting = 'same_kind')
    return array, mean, variance

In [67]:
def normalize_array(array):
    length = array.shape[0]
    array.astype(dtype=np.float32, copy = False)
    mean = array.sum(axis=0) / length
    array = np.subtract(array, mean, casting = 'same_kind')
    variance = (array ** 2).sum(axis=0) / length + 1e-27
    array = np.divide(array, variance, casting = 'same_kind')
    return array, mean, variance
    

In [38]:
img = read_images('data/train_images/', end_index = 500)

In [153]:
train_data = pd.read_csv('data/train.csv')
(train_rows, train_coloumns) = data.shape

In [146]:
test_data = pd.read_csv('data/test.csv')
(test_rows, test_columns) = data.shape

In [50]:
prediction.shape

(500, 1536)

In [42]:
def one_hot(array, num_options, option):
    for i in range(num_options):
        if option == i:
            array.append(1)
        else:
            array.append(0)

In [119]:
state_dict = dict({41336:0, 41325:1, 41367:2, 41401:3, 41415:4, 
                          41324:5, 41332:6, 41335:7, 41330:8, 
                          41380:9, 41327:10, 41345:11, 41342:12, 
                          41326:13, 41361:14})

{41336: 0, 41325: 1, 41367: 2, 41401: 3, 41415: 4, 41324: 5, 41332: 6, 41335: 7, 41330: 8, 41380: 9, 41327: 10, 41345: 11, 41342: 12, 41326: 13, 41361: 14}


In [148]:
def csv2array(data, num_rows, train = True):
    X_train = []
    if train:
        Y_train = []
    for row in range(num_rows):
        current_row = data.iloc[row]
        current_row_array = []

        one_hot(current_row_array, 2, current_row['Type']-1)

        current_row_array.append(current_row['Age'])

        one_hot(current_row_array, 307, current_row['Breed1'])
        one_hot(current_row_array, 307, current_row['Breed2'])
        one_hot(current_row_array, 3, current_row['Gender']-1)
        one_hot(current_row_array, 8, current_row['Color1'])
        one_hot(current_row_array, 8, current_row['Color2'])
        one_hot(current_row_array, 8, current_row['Color3'])
        one_hot(current_row_array, 5, current_row['MaturitySize'])
        one_hot(current_row_array, 4, current_row['FurLength'])
        one_hot(current_row_array, 3, current_row['Vaccinated']-1)
        one_hot(current_row_array, 3, current_row['Dewormed']-1)
        one_hot(current_row_array, 3, current_row['Sterilized']-1)
        one_hot(current_row_array, 4, current_row['Health'])
        current_row_array.append(current_row['Quantity'])
        current_row_array.append(current_row['Fee'])
        current_row_array.append(current_row['VideoAmt'])
        current_row_array.append(current_row['PhotoAmt'])
        state_id = current_row['State']
        one_hot(current_row_array, 15, state_dict[state_id])
        
        X_train.append(current_row_array)
        if train:
            temp = []
            one_hot(temp, 5, current_row['AdoptionSpeed'])
            Y_train.append(temp)
    X_train = np.array(X_train)
    if train:
        Y_train = np.array(Y_train)
        return X_train, Y_train
    else:
        return X_train

In [144]:
X_train, Y_train = csv2array(data, train_rows)

In [136]:
X_train, X_mean, X_variance = normalize_array(X_train)

In [149]:
X_test = csv2array(data, test_rows, train = False)

In [151]:
X_test -= X_mean
X_test /= X_variance

In [ ]:
cnn_outputs = []
for row in range(int(train_rows)):
    id  = train_data.iloc[row]['PetID']
    num_photos = train_data.iloc[row]['PhotoAmt']
    if num_photos != 0.0:
        photos_array = []
        for photo_num in range(int(num_photos)):
            img = cv2.imread('data/train_images/' + str(id) + '-' + str(int(photo_num + 1)) + '.jpg')
            img = cv2.resize(img, dsize=(299, 299))
            photos_array.append(img)
        photos_array = np.array(photos_array)
        print(photos_array.shape)
        photos_array, photos_mean, photos_variance = normalize_images(photos_array)
        photos_outputs = cnn.predict(photos_array)
        photos_array = photos_array.sum(axis=0) / num_photos
        cnn_outputs.append(photos_array)

(1, 299, 299, 3)
(2, 299, 299, 3)
(7, 299, 299, 3)
(8, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(9, 299, 299, 3)
(6, 299, 299, 3)
(2, 299, 299, 3)
(7, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(2, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(2, 299, 299, 3)
(8, 299, 299, 3)
(1, 299, 299, 3)
(7, 299, 299, 3)
(5, 299, 299, 3)
(19, 299, 299, 3)
(3, 299, 299, 3)
(11, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(7, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(13, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(6, 299, 299, 3)
(2, 299, 299, 3)
(11, 299, 299, 3)
(13, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(13, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(5, 299, 299, 3)
(15, 299, 299, 3)
(4, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(2, 299

(1, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(7, 299, 299, 3)
(1, 299, 299, 3)
(8, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(8, 299, 299, 3)
(4, 299, 299, 3)
(2, 299, 299, 3)
(6, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(1, 299, 299, 3)
(8, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(10, 299, 299, 3)
(2, 299, 299, 3)
(5, 299, 299, 3)
(11, 299, 299, 3)
(6, 299, 299, 3)
(6, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(10, 299, 29

(1, 299, 299, 3)
(5, 299, 299, 3)
(26, 299, 299, 3)
(27, 299, 299, 3)
(14, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(4, 299, 299, 3)
(6, 299, 299, 3)
(2, 299, 299, 3)
(2, 299, 299, 3)
(11, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(15, 299, 299, 3)
(1, 299, 299, 3)
(8, 299, 299, 3)
(5, 299, 299, 3)
(9, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(11, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(2, 299, 299, 3)
(12, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(4, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(2, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(4, 299, 299, 3)
(7, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(1, 299

(5, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(5, 299, 299, 3)
(30, 299, 299, 3)
(2, 299, 299, 3)
(4, 299, 299, 3)
(29, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(7, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(2, 299, 299, 3)
(5, 299, 299, 3)
(3, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(11, 299, 299, 3)
(2, 299, 299, 3)
(14, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(7, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(12, 299, 299, 3)
(14, 299, 299, 3)
(18, 299, 299, 3)
(3, 299, 299, 3)
(4, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(15, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(7, 299, 299, 3)
(1, 299, 299, 3)
(3, 299, 299, 3)
(6, 29

(14, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(7, 299, 299, 3)
(2, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(6, 299, 299, 3)
(1, 299, 299, 3)
(2, 299, 299, 3)
(8, 299, 299, 3)
(5, 299, 299, 3)
(8, 299, 299, 3)
(2, 299, 299, 3)
(5, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(2, 299, 299, 3)
(6, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(2, 299, 299, 3)
(8, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(3, 299, 299, 3)
(5, 299, 299, 3)
(5, 299, 299, 3)
(3, 299, 299, 3)
(2, 299, 299, 3)
(4, 299, 299, 3)
(6, 299, 299, 3)
(5, 299, 299, 3)
(4, 299, 299, 3)
(4, 299, 299, 3)
(7, 299, 299, 3)
(5, 299, 299, 3)
(6, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(1, 299, 299, 3)
(5, 299, 299, 3)
(2, 299, 299, 3)
(13, 299, 299, 3)
(1, 299, 299, 3)
(1, 299, 299, 3)
(30, 299, 299, 3)
(5, 299, 299, 3)
(9, 299, 29

In [137]:
fc_input = Input(shape=(685,))
current_activation = Dense(250)(fc_input)
current_activation = BatchNormalization()(current_activation)
current_activation = Activation(activation='relu')(current_activation)
layers =  [250, 100, 50, 25, 10]
for layer in layers:
    current_activation = Dense(layer)(current_activation)
    current_activation = BatchNormalization()(current_activation)
    current_activation = Activation(activation='relu')(current_activation)
current_activation = Dense(5)(current_activation)
output = Activation(activation='softmax')(current_activation)

In [138]:
fc_model = Model(inputs=fc_input, outputs=output)
fc_model.compile(optimizer=Adam(lr=0.0008, decay=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [139]:
fc_model.fit(X_train, Y_train, epochs = 100, batch_size = 128)

Epoch 1/100
14993/14993 [==============================] - 10s 641us/step - loss: 1.6148 - acc: 0.2402
Epoch 2/100
14993/14993 [==============================] - 2s 110us/step - loss: 1.4773 - acc: 0.3189
Epoch 3/100
14993/14993 [==============================] - 2s 116us/step - loss: 1.4439 - acc: 0.3378
Epoch 4/100
14993/14993 [==============================] - 2s 114us/step - loss: 1.4252 - acc: 0.3444
Epoch 5/100
14993/14993 [==============================] - 2s 118us/step - loss: 1.4140 - acc: 0.3544
Epoch 6/100
14993/14993 [==============================] - 2s 111us/step - loss: 1.4081 - acc: 0.3519
Epoch 7/100
14993/14993 [==============================] - 2s 112us/step - loss: 1.4045 - acc: 0.3563
Epoch 8/100
14993/14993 [==============================] - 2s 112us/step - loss: 1.3974 - acc: 0.3598
Epoch 9/100
14993/14993 [==============================] - 2s 111us/step - loss: 1.3933 - acc: 0.3634
Epoch 10/100
14993/14993 [==============================] - 2s 113us/step - loss:

14993/14993 [==============================] - 2s 114us/step - loss: 1.2476 - acc: 0.4496
Epoch 81/100
14993/14993 [==============================] - 2s 116us/step - loss: 1.2554 - acc: 0.4462
Epoch 82/100
14993/14993 [==============================] - 2s 118us/step - loss: 1.2467 - acc: 0.4469
Epoch 83/100
14993/14993 [==============================] - 2s 117us/step - loss: 1.2440 - acc: 0.4560
Epoch 84/100
14993/14993 [==============================] - 2s 116us/step - loss: 1.2389 - acc: 0.4533
Epoch 85/100
14993/14993 [==============================] - 2s 114us/step - loss: 1.2396 - acc: 0.4556
Epoch 86/100
14993/14993 [==============================] - 2s 112us/step - loss: 1.2332 - acc: 0.4607
Epoch 87/100
14993/14993 [==============================] - 2s 113us/step - loss: 1.2311 - acc: 0.4551
Epoch 88/100
14993/14993 [==============================] - 2s 114us/step - loss: 1.2292 - acc: 0.4615
Epoch 89/100
14993/14993 [==============================] - 2s 112us/step - loss: 1.23